In [2]:
#https://news.mail.ru/

In [3]:
import requests
from lxml import html
import time
import pymongo
from pprint import pprint
from pymongo.errors import *

In [4]:
###Creating the database
client = pymongo.MongoClient('127.0.0.1', 27017)
db = client['db_news']
news = db.news

In [5]:
#Function to avoid duplicates and add data to db
def db_add(db, docs):
    for doc in docs:
        try:
            db.insert_one(doc)
        except:
            pass

In [22]:
#Data to enter the needed profession in
url = 'https://news.mail.ru/'
headers = {'User-Agent': 'Chrome/98.0.4758.102'}
response = requests.get(url, headers=headers)
root = html.fromstring(response.text)

In [67]:
titles_in_pic = root.xpath("//span[contains(@class,'photo__title')]/text()") #first 3
titles_in_list = root.xpath("//a[@class='list__text']/text()")
titles_in_list2 = root.xpath("//a[contains(@class,'link_flex')]/span/text()")
titles_with_pic = root.xpath("//span[@class='newsitem__title-inner']/text()")
len(titles_in_pic) + len(titles_in_list) + len(titles_in_list2) + len(titles_with_pic)

39

In [68]:
links_in_pic = root.xpath("//span[contains(@class,'photo__title')]/../../@href") #first 3
links_in_list = root.xpath("//a[@class='list__text']/@href")
links_in_list2 = root.xpath("//a[contains(@class,'link_flex')]/@href")
links_with_pic = root.xpath("//span[@class='newsitem__title-inner']/../@href")

In [69]:
source_with_pic = root.xpath("//span[@class='newsitem__param']/text()")
date_with_pic = root.xpath("//span[contains(@class,'js-ago')]/@datetime")

In [70]:
blocks = [[titles_in_pic, links_in_pic],
         [titles_in_list, links_in_list], 
         [titles_in_list2, links_in_list2], 
         [titles_with_pic, links_with_pic, source_with_pic, date_with_pic]]

In [71]:
news_list = []

for block in blocks:
    for i in range(len(block[0])):    
        news_item = {}

        news_item['title'] = block[0][i]
        news_item['link'] = block[1][i]
        try:
            news_item['source'] = block[2][i]
            news_item['date'] = block[3][i]
        except:
            news_item['source'] = None
            news_item['date'] = None

        news_list.append(news_item)

In [ ]:
news_in_pic = root.xpath("//table[@class='daynews__inner']//div[contains(@class,'daynews__item')]/text()")
news_in_list = root.xpath("//li[contains(@class,'list__item')]/text()")
news_with_pic = root.xpath("//span[@class='cell']/text()")